In [60]:
import pandas as pd
import os
import subprocess
import openpyxl

In [61]:
current_dir = os.getcwd()
lua_file_path = os.path.join(current_dir,"luascript", "prasejson.lua")

In [62]:
def get_json(s):
    result = subprocess.run(["lua", f"{lua_file_path}" ,s], capture_output=True, text=True)
    return result.stdout

def get_dict():
    en_str = get_json("en")
    zh_str = get_json("zh")
    en_dict = eval(en_str)
    zh_dict = eval(zh_str)
    return en_dict,zh_dict

In [31]:
def get_miss_key(prefix,e_dict,z_dict):
    for key, item in e_dict.items():
        if key not in z_dict.keys():
            print(f"miss: {prefix}{key}")
        else:
            if isinstance(item, dict):
                get_miss_key(prefix  + f"{key}->",e_dict[key],z_dict[key])
    

## 检查 en中有而 zh中没有的key

In [32]:
en_dict,zh_dict = get_dict()
get_miss_key("",en_dict,zh_dict)

## 合并（更新）CSV中的内容到当前lua

In [53]:
def dict_to_lua_table(d, indent=0):
    """将 Python 字典转换为 Lua table 格式的字符串，并添加缩进和换行。"""
    indent_str = "\t" * (indent+1)
    next_indent_str = "\t" * (indent + 2)
    
    if isinstance(d, dict):
        items = []
        for key, value in d.items():
            # 递归处理键值对，key 必须是字符串，并且放在 Lua 格式的括号中
            if "-" in key or "." in key or " " in key :
                key = f"[\"{key}\"]"
            items.append(f'{next_indent_str}{key} = {dict_to_lua_table(value, indent + 1)}')
        return "{\n" + ",\n".join(items) + "\n" + indent_str + "}"
    
    elif isinstance(d, list):
        items = [] 
        for item in d:
            if isinstance(item, list):
                items.append(f'{next_indent_str}{dict_to_lua_table(item, indent + 1)}') 
            else:
                items.append(f'{next_indent_str}"{item}"')
        
        return "{\n" + ",\n".join(items) + "\n" + indent_str + "}"
    
    elif isinstance(d, str):
        # 处理字符串类型，保留双引号
        return f'"{d}"'
    
    else:
        # 处理其他类型，转换为字符串格式
        return str(d)
    
def dict2lua(dict):
    table =dict_to_lua_table(dict)
    return "return " +table

def sort_dict(d):
    if isinstance(d, dict):
        return {k: sort_dict(v) for k, v in sorted(d.items())}
    return d

In [54]:
def merge_excel_2_lua(moded_csv, change_name = True, change_discription = True , out_name = "o_zh_CN.lua"):
    moded_df = pd.read_excel(moded_csv , engine= 'openpyxl')
    # 读取原来的json
    en_dict,zh_dict = get_dict()
    # 读取csv
    # 更具csv内容更改json
    filtered_df = moded_df[(moded_df['修改后的中文名称'].notna()) | (moded_df['修改后的中文介绍'].notna())]

    for index, row in filtered_df.iterrows():
        # 访问每一行的数据，例如：
        kind = row['kind']
        key = row['key']
        chinese_name = row['修改后的中文名称']
        introduction = row['修改后的中文介绍']
        
        origin_name = row['中文名称']
        origin_text = row['中文介绍']
        
        if change_name and  pd.notna(row['修改后的中文名称']):
            zh_dict["descriptions"][kind][key]["name"] = chinese_name
            
        if change_discription and pd.notna(row['修改后的中文介绍']):
            zh_dict["descriptions"][kind][key]["text"] =  str(introduction).split("\n")
            
    res = (dict2lua(sort_dict(zh_dict)))
    
    with open(out_name, 'w', encoding='utf-8') as f:
        f.write(res)
        f.close()
    # 更具json生成lua

In [55]:
merge_excel_2_lua('翻译勘误.xlsx' , False , True,)

c:\ProgramData\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## 导出当前lua到csv

In [63]:
def lua_2_csv():
    en_dict,zh_dict = get_dict()
    csv_data = []
    for kind in zh_dict["descriptions"].keys():
        for key in zh_dict["descriptions"][kind].keys():
            d = {
                "kind" : kind ,
                "key" : key ,
                "英文名称" :  en_dict["descriptions"][kind][key]["name"] if key in en_dict["descriptions"][kind].keys() else '',
                "中文名称" :  '未翻译',
                "修改后的中文名称" :  '' ,
                "英文介绍" :  "\n".join(en_dict["descriptions"][kind][key]["text"])  if key in en_dict["descriptions"][kind].keys() else '',
                "中文介绍" :  '未翻译'  ,
                "修改后的中文介绍" :  '' ,
            }
            if key in zh_dict["descriptions"][kind].keys():
                d["中文名称"] = zh_dict["descriptions"][kind][key]["name"] 
                d["中文介绍"] = "\n".join(zh_dict["descriptions"][kind][key]["text"])
            
            csv_data.append(d)
    df = pd.DataFrame(csv_data)
    df.to_csv('out.csv', index=False, encoding="utf-8")

In [65]:
lua_2_csv()
